# 04-7: Hugging Face diffusers library intro

This Colab requires T4 GPU.

In [ ]:
!pip install diffusers accelerate

In [ ]:
from diffusers import DiffusionPipeline

In [ ]:
# TODO: Implement a DiffusionPipeline for "runwayml/stable-diffusion-v1-5"

In [ ]:
# TODO: Run the previous pipeline and save the image

## Use a diffusion model locally

In [ ]:
# TODO: Implement a LOCAL DiffusionPipeline for "runwayml/stable-diffusion-v1-5"

## Use different schedulers

In [ ]:
# TODO: Generate an image using a different scheduler (EulerDiscreteScheduler)

## Models

In [ ]:
# TODO: Pick UNet2DModel from repo "google/ddpm-cat-256"

In [ ]:
# TODO: Generate a noisy image

In [ ]:
# TODO: For inference, pass a noise image and a timestep, which indicates how noisy the input image is.

## Schedulers

In [ ]:
from diffusers import DDPMScheduler

In [ ]:
# TODO: Set the `DDPMScheduler` scheduler to generate an image with previous model. Reuse the code below

In [ ]:
# To predict a slightly less noisy image, pass the following to the scheduler’s step() method: model output, timestep, and current sample.

less_noisy_sample = scheduler.step(model_output=noisy_residual, timestep=2, sample=noisy_sample).prev_sample
less_noisy_sample.shape

In [ ]:
# Function that postprocesses and displays the denoised image as a PIL.Image:
import PIL.Image
import numpy as np


def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

In [ ]:
model.to("cuda")
noisy_sample = noisy_sample.to("cuda")

In [ ]:
# Create a denoising loop that predicts the residual of the less noisy sample, and computes the less noisy sample with the scheduler:

import tqdm

sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute less noisy image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 50 == 0:
        display_sample(sample, i + 1)